In [1]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import minmax_scale ,StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
#importing the datasets
cc = pd.read_excel("CreditConsumptionData.xlsx")
cb = pd.read_excel("CustomerBehaviorData.xlsx")
cd = pd.read_excel("CustomerDemographics.xlsx")

In [3]:
Data = pd.merge(left=cc,right=cb,how="left",on="ID")

In [4]:
final_dataframe = pd.merge(left=Data,right=cd,how="left",left_on="ID",right_on="ID")

In [ ]:
heatmap = final_dataframe.drop(columns = ['Income','gender','account_type','loan_enq'])

In [ ]:
#plt.figure(figsize=(10, 8))
sns.heatmap(heatmap, annot=True, cmap='magma', fmt=".2f", linewidths=0.5)
plt.show()

In [5]:
#final merge of the datasets
final_data = final_dataframe[-final_dataframe.cc_cons.isna()]

In [6]:
#A split of the dataset with the missing values for predicting values for customers where target variable having missing values. 
final_test = final_dataframe[final_dataframe.cc_cons.isna()]

In [7]:
#treating missing values
def missing_treat(x):
    if ((x.dtype=='float') or (x.dtype=='int')):
        x = x.fillna(x.median())
    elif x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    else:
        x
    return(x)

In [8]:
final_data = final_data.apply(missing_treat)

In [9]:
#removing outliers
def outlier_var(x):
    if ((x.dtype=='float') or (x.dtype=='int')):
        x= x.clip(lower = x.quantile(0.01), upper = x.quantile(0.99))
    else:
        x
    return x

In [10]:
final_data = final_data.apply(outlier_var)

In [11]:
#OneHot Encoding
final_data = pd.get_dummies(final_data, columns = ['account_type', 'loan_enq','gender'], drop_first=True)

In [12]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
encode=LabelEncoder()

In [13]:
final_data = final_data.drop(columns=["Income"])

In [14]:
x = final_data.drop(columns=["cc_cons"])
y = final_data["cc_cons"]

In [16]:
#Transforming X variables
power = PowerTransformer(method='yeo-johnson', standardize=True)
data_trans = power.fit_transform(x)

In [17]:
trans_x = pd.DataFrame(data_trans,columns=x.columns,index=x.index)

In [23]:
trans_x

,ID,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction,account_type_saving,gender_M,income
0,0.496229,0.826884,-1.715682,0.165817,1.801907,0.808405,1.019743,-1.341058,0.261033,-0.844590,...,-0.528495,-0.506160,0.485522,-1.685359,0.510793,-1.601713,1.062561,-0.422607,0.398433,0.739725
1,1.235568,0.576767,-1.061748,0.769716,-1.981929,-0.390962,-1.030588,-1.789104,-0.808113,-0.655068,...,0.840001,-0.506160,0.485522,0.227826,-1.714818,-1.601713,0.572265,-0.422607,0.398433,-1.800800
2,-0.313372,-0.482384,-0.288832,-1.202962,0.043317,1.170006,0.380836,0.896241,0.445965,1.378246,...,1.333903,0.883907,1.020643,-1.685359,0.926718,0.624331,-1.545996,-0.422607,0.398433,0.739725
3,-1.582405,1.014963,-0.882885,0.340242,0.755287,0.875961,1.162778,1.024578,-1.924628,0.556136,...,-0.054311,-1.356143,-0.819558,0.227826,1.211179,0.624331,-1.316780,-0.422607,-2.509835,0.739725
4,1.083268,-0.250768,0.452410,-0.554205,-0.530958,-0.962192,0.604865,1.163202,0.798020,1.111789,...,0.825298,-0.624734,0.294121,-0.468967,0.926718,0.624331,-0.690442,-0.422607,0.398433,0.739725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.422307,0.044558,-0.893105,-0.130674,0.030591,-0.000504,0.920456,-1.043417,0.552524,-1.536565,...,2.239279,0.797117,0.345212,1.181361,0.264662,0.624331,1.531944,-0.422607,-2.509835,-1.181775
14996,0.154144,0.996464,0.378179,0.547474,0.598150,-0.927594,-1.070618,-1.789104,0.500665,-1.397157,...,0.140337,0.294628,-1.445369,1.181361,-0.271402,0.624331,-1.790810,-0.422607,0.398433,0.739725
14997,-1.425862,-0.747068,-1.074714,0.712279,-1.584048,1.766662,-0.463457,1.068199,0.153644,0.771693,...,-0.188979,-0.290366,-1.106861,1.181361,-0.600086,0.624331,-0.496277,-0.422607,0.398433,0.739725
14998,1.341890,2.304122,0.485698,1.449217,1.949153,1.030998,-0.925357,-0.389559,1.770604,0.422891,...,0.837976,2.378433,1.545443,1.481469,-0.266808,0.624331,-1.545996,-0.422607,0.398433,0.739725


In [18]:
#Transforming Y variables
trans_y = np.log(y)

## Feature selection for selecting the best values

In [19]:
from sklearn.feature_selection import SelectKBest, f_regression

In [20]:
SKB = SelectKBest(f_regression,k=20).fit(trans_x,trans_y)
trans_x.columns[SKB.get_support()].values

array(['dc_cons_apr', 'cc_cons_may', 'dc_cons_jun', 'cc_count_apr',
       'cc_count_may', 'dc_count_apr', 'card_lim', 'investment_3',
       'credit_amount_apr', 'debit_count_apr', 'max_credit_amount_apr',
       'credit_amount_jun', 'max_credit_amount_jun', 'emi_active', 'age',
       'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'Avg_days_between_transaction', 'income'], dtype=object)

In [21]:
columns = ['dc_cons_apr', 'cc_cons_may', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 
           'dc_count_apr', 'card_lim', 'investment_3', 'credit_amount_apr', 'debit_count_apr',
           'max_credit_amount_apr', 'credit_amount_jun', 'max_credit_amount_jun', 'emi_active', 
           'age', 'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code', 'Avg_days_between_transaction', 'income']

trans_xf = trans_x[columns]


In [22]:
#splitting the data into train and test values
x_train, x_test, y_train, y_test  = train_test_split(trans_xf, trans_y, test_size=0.2, random_state=1234)

In [23]:
#standard transformation of values
sc = StandardScaler()
train_X_std = sc.fit_transform(x_train)
test_X_std = sc.fit_transform(x_test)

In [31]:
train_X_std

array([[ 0.26826459, -0.18235857,  0.26794621, ...,  0.57351187,
         1.22710596,  0.74167876],
       [ 0.83350858,  0.40219015,  0.89851905, ..., -0.39989287,
         1.53917781, -1.17494947],
       [ 0.12385771,  0.78738806,  1.42356185, ...,  0.66187262,
         0.90635091, -1.17494947],
       ...,
       [ 0.26812395,  1.32893831,  0.89208061, ...,  0.66187262,
        -0.89365359,  0.74167876],
       [-2.01003754, -0.22023613, -0.44594162, ...,  1.18683005,
         0.40595887,  0.74167876],
       [ 0.48036293, -0.06350164, -0.80438919, ..., -1.70816324,
        -0.89365359,  0.74167876]])

In [24]:
train_X_std_1 = pd.DataFrame(train_X_std, columns = x_train.columns)

In [36]:
train_X_std_1

,dc_cons_apr,cc_cons_may,dc_cons_jun,cc_count_apr,cc_count_may,dc_count_apr,card_lim,investment_3,credit_amount_apr,debit_count_apr,max_credit_amount_apr,credit_amount_jun,max_credit_amount_jun,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,Avg_days_between_transaction,income
0,0.268265,-0.182359,0.267946,-0.816979,-0.211180,-0.945876,1.402025,0.535476,0.684844,-0.375478,1.499723,0.093931,0.199424,0.181366,1.247890,1.445482,-0.476644,0.573512,1.227106,0.741679
1,0.833509,0.402190,0.898519,-1.334181,-1.381343,-0.066311,1.252493,1.348303,0.318957,0.590567,-0.581779,-0.204934,-0.210062,1.532814,0.418648,-0.234357,-1.254388,-0.399893,1.539178,-1.174949
2,0.123858,0.787388,1.423562,0.025731,-0.710111,-0.476232,0.417807,0.035062,2.209329,-1.618010,-0.852422,2.232743,-0.083406,-0.000028,0.993202,0.933677,1.475454,0.661873,0.906351,-1.174949
3,-0.047089,0.859387,-0.479378,0.358509,-0.538285,-0.945876,-0.004448,-0.470584,-0.851125,0.735165,1.388260,0.402659,-0.480012,0.282532,-1.735211,-1.076658,1.175091,-1.578730,-0.307897,0.741679
4,0.833218,-1.780691,0.568110,-0.280649,-1.200274,-0.945876,1.049873,0.612896,-0.147987,0.950613,-2.236946,1.554885,1.387804,0.199069,-0.890270,-1.048322,-0.852530,1.069453,0.906351,-1.792405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,-0.368740,-0.686741,-0.331539,-0.110810,0.551209,-0.066311,0.601420,-0.162361,-0.932950,0.512934,-0.476913,-0.434655,0.287970,0.092187,-1.190772,-0.206920,1.175091,1.394519,1.227106,-1.792405
11996,-0.485243,1.067200,-1.010760,-0.816979,0.499332,-1.337449,0.290761,-1.494863,0.202279,0.590567,0.633905,-0.822352,-0.659304,0.086614,0.916578,1.184802,0.549054,-0.271480,1.067888,-1.174949
11997,0.268124,1.328938,0.892081,1.159503,0.386709,0.636066,0.123418,0.798474,0.078790,1.393324,-0.620271,0.697150,0.148520,0.585291,2.371823,2.660567,-0.476644,0.661873,-0.893654,0.741679
11998,-2.010038,-0.220236,-0.445942,0.915778,0.356438,-0.314754,-0.924619,0.453567,0.141660,0.640252,-0.008719,0.362611,1.282004,-0.226602,0.916578,1.422238,-1.254388,1.186830,0.405959,0.741679


In [25]:
test_X_std_1 = pd.DataFrame(test_X_std, columns = x_test.columns)

In [39]:
test_X_std_1

,dc_cons_apr,cc_cons_may,dc_cons_jun,cc_count_apr,cc_count_may,dc_count_apr,card_lim,investment_3,credit_amount_apr,debit_count_apr,max_credit_amount_apr,credit_amount_jun,max_credit_amount_jun,emi_active,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,Avg_days_between_transaction,income
0,-0.258025,-1.181359,-0.044170,-0.311331,-0.260163,0.020147,-1.931198,1.051737,0.494736,-0.811201,0.090443,-0.355240,-2.391282,-0.495592,-2.179590,-2.264854,-0.083542,0.100601,1.041854,0.731966
1,0.202202,0.617134,-0.004693,-0.849846,-0.050781,1.090300,0.286097,1.570698,-0.236551,-0.885766,-0.367503,0.731653,-0.233771,1.546937,0.828990,0.823500,-1.213480,-0.252423,-0.306771,0.731966
2,-0.340057,0.018726,0.267817,-0.849846,-1.041046,-0.948742,1.265229,0.281241,1.033488,-0.390595,0.066339,0.028791,-0.860170,0.948876,0.514726,1.050685,0.899705,-1.742504,-1.525594,0.731966
3,0.268742,0.528200,0.946461,0.251210,-0.701425,0.020147,0.416915,-1.569608,-0.616959,1.107261,-0.099774,0.694078,-0.375522,-2.212294,1.373218,0.812198,-0.083542,-0.266466,0.883506,0.731966
4,0.249829,1.016637,0.332209,0.520954,0.698820,0.616722,0.087057,0.827506,0.649001,-0.811201,0.022528,-0.180144,-0.219018,0.406967,1.107435,-0.190073,-1.213480,0.095988,-0.125900,-1.835208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-0.054855,-1.081687,0.221344,-0.534256,-1.041046,-0.948742,-0.586178,-1.061150,-0.772129,2.171077,0.287022,1.116707,-0.340414,0.862299,0.514726,0.145230,1.506224,0.275291,1.503840,0.731966
2996,-0.990826,-0.830612,0.286353,1.530621,-1.371537,0.853217,-0.810221,1.206921,-0.490471,1.107261,1.769509,0.051480,0.736207,-1.685047,-0.873194,-0.507825,0.899705,-1.660750,1.503840,0.731966
2997,0.256506,0.938483,-1.277212,-1.070522,0.120895,1.199744,-0.977949,-0.876866,2.931590,-1.495168,-0.729232,-2.375469,-0.055456,0.739181,-0.608268,-1.307813,-0.083542,-0.492352,-0.492418,0.731966
2998,2.447664,1.049544,-0.268372,-0.534256,-0.098965,0.020147,0.492073,0.623658,0.308718,1.069536,1.294036,0.956051,-0.497523,-0.218285,-0.873194,0.145230,-1.651532,-0.421512,-1.085911,0.731966


### Predicted values for customers where target variable having missing values 

In [26]:
final_test.drop(columns=["cc_cons"],inplace=True)

In [27]:
def missing_treat(x):
    if ((x.dtype=='float') or (x.dtype=='int')):
        x = x.fillna(x.median())
    elif x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    else:
        x
    return(x)

In [28]:
final_test = final_test.apply(missing_treat)

In [29]:
final_test = pd.get_dummies(final_test, columns = ['account_type', 'loan_enq','gender'], drop_first=True)

In [30]:
final_test["income"]=encode.fit_transform(final_test["Income"])
final_test=final_test.drop(columns=["Income"])

In [31]:
power = PowerTransformer(method='yeo-johnson', standardize=True)
data_trans = power.fit_transform(final_test)

In [32]:
missing_test_x = pd.DataFrame(data_trans,columns=final_test.columns,index=final_test.index)

In [33]:
missing_test_x = missing_test_x[['dc_cons_apr', 'cc_cons_may', 'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 
           'dc_count_apr', 'card_lim', 'investment_3', 'credit_amount_apr', 'debit_count_apr',
           'max_credit_amount_apr', 'credit_amount_jun', 'max_credit_amount_jun', 'emi_active', 
           'age', 'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code', 'Avg_days_between_transaction', 'income']]

In [54]:
rf = RandomForestRegressor(n_estimators= 1000, min_samples_split= 20, max_features= 5, max_depth= 15,n_jobs=-1)
rf.fit(x_train, y_train)

RandomForestRegressor(max_depth=15, max_features=5, min_samples_split=20,
                      n_estimators=1000, n_jobs=-1)

In [55]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Calculate RMSPE
def rmspe(y_test, y_pred):
    return np.sqrt(np.mean(((y_test - y_pred) / (y_test))**2))

y_pred = rf.predict(x_test)

# Calculate RMSPE
rmspe_value = rmspe(y_test, y_pred)
print(f'RMSPE: {rmspe_value:}')


RMSPE: 0.17486190470561536


In [56]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Calculate RMSPE
def rmspe(y_train, y_pred):
    return np.sqrt(np.mean(((y_train - y_pred) / (y_train))**2))

y_pred = rf.predict(x_train) 

# Calculate RMSPE
rmspe_value = rmspe(y_train, y_pred)
print(f'RMSPE: {rmspe_value:}')


RMSPE: 0.15068352301501436


In [37]:
ada = AdaBoostRegressor(n_estimators=50, learning_rate=1.0, loss='linear')
ada.fit(x_train, y_train)

AdaBoostRegressor()

In [40]:
gb = GradientBoostingRegressor(subsample= 0.1, n_estimators= 400, max_features= 'log2', loss= 'huber', learning_rate= 0.1, criterion= 'squared_error')
gb.fit(x_train, y_train)

GradientBoostingRegressor(criterion='squared_error', loss='huber',
                          max_features='log2', n_estimators=400, subsample=0.1)

In [42]:
svr = SVR(kernel= 'rbf', gamma= 0.1, C= 1000)
svr.fit(x_train, y_train)

SVR(C=1000, gamma=0.1)

In [45]:
dt = DecisionTreeRegressor(splitter= 'best', max_features= 'log2', criterion= 'poisson', ccp_alpha= 0)
dt.fit(x_train, y_train)

DecisionTreeRegressor(ccp_alpha=0, criterion='poisson', max_features='log2')

In [48]:
knn = KNeighborsRegressor(algorithm= 'ball_tree', leaf_size= 2, metric= 'manhattan', n_neighbors= 4, p= 2, weights= 'uniform')
knn.fit(x_train, y_train)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=2, metric='manhattan',
                    n_neighbors=4)

In [51]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [57]:
#Predict on Test Data
pred = rf.predict(missing_test_x)

In [58]:
pred_y = pd.DataFrame(pred)

In [59]:
y_pred = np.exp(pred_y)
y_pred.columns = ['cc_cons']
y_pred.index = final_test.index
y_pred["ID"] = final_test.ID

In [60]:
y_pred

,cc_cons,ID
15000,15081.195486,17591
15001,14104.977708,13541
15002,13292.012304,13431
15003,20621.696659,8687
15004,12833.871220,14727
...,...,...
19995,19963.842562,12512
19996,15681.177549,14224
19997,14185.249502,18740
19998,15400.222607,18134
